In [ ]:
# Load all 4 Datasets
# Here, I have taken 2 csv's and one xml document to be merged together. The csv's taken are for dlr_pitch data and 
# dcc_pitch data. Fingal data has been downloaded first in xml form, then I parsed it to a pandas dataframe using xml.etree 
# as my parser. Before starting off, we import pandas and numpy since they get used extensively throughout the code

import pandas as pd
import numpy as np

# reading in dlr_pitch data
dlr_pitch = pd.read_csv('loading the csv file of dlr_pitch')
print (dlr_pitch)

# reading in dcc_pitch data
dcc_pitch = pd.read_csv('loading the other csv of dcc_pitch data')
print(dcc_pitch)


In [ ]:
# In the dlr_pitch dataset, the location column has sparsely populated values. The Location value is just stated once
## Fill all rows of blank location with previous row values
dlr_pitch['Location'] = dlr_pitch['Location'].fillna(method='ffill')
print(dlr_pitch)


In [ ]:
# Importing the fingal dataset in xml format and converting to a usable pandas dataframe

import xml.etree.cElementTree as et
xml_file = et.parse("load the xml file of fcc playing pitches")

def findval(node):
    if node is not None:
       return node.text
    else:
       return None

def parsing():
    root = xml_file.getroot()
    column_name = ['Pitch_type','Name','loc','lat','long']
    fcc_data = pd.DataFrame(columns = column_name)

    
    for node in root.findall('./Playing_Pitches-table/Playing_Pitches'):
        Pitch_type = node.find('FACILITY_TYPE')
        Name = node.find('FACILITY_NAME')
        loc = node.find('LOCATION')
        lat = node.find('LAT')
        long = node.find('LONG')
        
        fcc_data = fcc_data.append(
            pd.Series([findval(Pitch_type),findval(Name),findval(loc),
                        findval(lat),findval(long)],index = column_name),ignore_index=True)
    part_data = pd.DataFrame(fcc_data).copy(deep = True)
    return part_data
    
fcc_data = parsing()
print(fcc_data)        
        

In [ ]:
# check for duplicates in dlr, dcc and fcc data
dlr_pitch
dlr_pitch = dlr_pitch.drop_duplicates(subset=list(dlr_pitch), keep=False)
#dlr_pitch
print(len(dlr_pitch))

## removing duplicates from dcc
#len(dcc_pitch)
dcc = dcc_pitch.drop_duplicates(subset=list(dcc_pitch), keep=False)
print(len(dcc))

## fcc
print(len(fcc_data))
fcc = fcc_data.drop_duplicates(subset=list(fcc_data), keep=False)
# fcc
## first changing the col names in fcc to latitude and longitude instead of lat, long
list(fcc)
fcc.columns = ['TypeOfPitch','Name','Location_fcc','Latitude','Longitude']
# fcc




In [ ]:
# Checking lengths of dlr_pitch, fcc data 
list(dlr_pitch)
list(fcc)

In [ ]:
# we check the dtypes of the two tables
print (fcc['Latitude'].dtypes)
print (fcc['Longitude'].dtypes)
print (dlr_pitch['Latitude'].dtypes)
print (dlr_pitch['Longitude'].dtypes)

# we see that in fcc - the data type of latitude and longitude is object whereas in dlr_pitch it is float64
# thus we convert both to float64
fcc['Latitude'] = fcc['Latitude'].astype(np.float64)
fcc['Longitude'] = fcc['Longitude'].astype(np.float64)
dlr_pitch['Longitude'] = dlr_pitch['Longitude'].astype(np.float64)
dlr_pitch['Latitude'] = dlr_pitch['Latitude'].astype(np.float64)

# Now we sanity check the fields
print (fcc['Latitude'].dtypes)
print (fcc['Longitude'].dtypes)
print (dlr_pitch['Latitude'].dtypes)
print (dlr_pitch['Longitude'].dtypes)


In [ ]:
dlr_pitch = dlr_pitch.round({'Latitude': 5, 'Longitude': 5})
dlr_pitch['Lat-Long'] = list(zip(dlr_pitch.Latitude, dlr_pitch.Longitude))
print(dlr_pitch.head(n=10))

fcc = fcc.round({'Latitude': 5, 'Longitude': 5})
fcc['Lat-Long'] = list(zip(fcc.Latitude, fcc.Longitude))
print(fcc.head(n=10))

## merge dlr and fcc pitch data on lat and long
dlr_fcc = dlr_pitch.merge(fcc, on=['Lat-Long'], how ='outer')
print(dlr_fcc.head(n=6))


In [ ]:
## Rename columns of merged dataset to recognize which tables have given the columns
dlr_fcc = dlr_fcc.rename(columns = {'Location_x' : 'Loc_dlr'})

In [ ]:
## Checking dcc_pitch'PARK','AREA','CLUBNAME','LEAGUE'
dcc_pitch = dcc_pitch.loc[:,['PARK','AREA','CLUBNAME','LEAGUE']]
dcc_pitch

base_data = pd.read_csv('# loading a dataset with information on all town lat-long ##')
base_data

## filtered and extracted only dublin data
base_data = base_data.loc[base_data['County'] == 'DUBLIN']
base_data

## select imp columns and drop the rest
base_data = base_data.loc[:,['X','Y','County','Contae','Local_Government_Area','English_Name','Irish_Name','Validated_By']]

## rename x and y in base_data to Longitude and Latitude 
base_data = base_data.rename(columns = {'Y':'Latitude', 'X': 'Longitude'})

In [ ]:
## on finding suitable merging column from base_data for dcc

base_data.head(n=5)
base_data.dtypes
dcc_pitch.dtypes

In [ ]:
# Converting column English_name to Location in base_data for easy merging
dcc_pitch.head(n=5)
base_data.head(n=6)
base_data['Location'] = base_data['English_Name'].str.lower()
dcc_pitch['Location'] = dcc_pitch['PARK'].str.lower()


In [ ]:
# Merging dcc and base_data
base_data.head(n=6)
dcc_pitch.head(n=5)
dcc_final = pd.merge(base_data,dcc_pitch, on = 'Location', how = 'right')
dcc_final


In [ ]:
## Sanity Check and checking length of data
print(len(dcc_final))
print(len(dlr_pitch))
len(fcc) 

print (fcc['Latitude'].dtypes)
print (fcc['Longitude'].dtypes)
print (dcc_final['Latitude'].dtypes)
print (dcc_final['Longitude'].dtypes)


In [ ]:
## Merging all datasets together
dcc_fcc = pd.merge(dcc_final, fcc, on = ['Latitude','Longitude'], how = 'outer')

dcc_fcc_dlr = pd.merge(dcc_fcc,dlr_pitch, on = ['Latitude','Longitude'], how = 'outer' )
dcc_fcc_dlr

In [ ]:
merged_data = dcc_fcc_dlr[dcc_fcc_dlr['Latitude'].notnull()]
merged_data.head(n=2)

keep the important columns from the merged dataset
dcc_fcc_dlr = dcc_fcc_dlr.loc[:,['Longitude','Latitude','County','Contae','PARK','AREA',
                       'CLUBNAME','LEAGUE','TypeOfPitch','Name','Location_fcc',
                       'Number','Size','Local_Government_Area','English_Name','Irish_Name','Validated_By']]
dcc_fcc_dlr = dcc_fcc_dlr.drop_duplicates(subset=list(dcc_fcc_dlr), keep=False)

# fill all rows of county with dublin since all the data is for pitches around dublin
dcc_fcc_dlr['County'] = dcc_fcc_dlr['County'].fillna(method='ffill')
dcc_fcc_dlr['Contae'] = dcc_fcc_dlr['Contae'].fillna(method='ffill')
dcc_fcc_dlr



In [ ]:
# Combine fields such as park, location_fcc, name to generate one location field:
# First, I make all entries in these columns to lowercase
dcc_fcc_dlr = dcc_fcc_dlr.drop('PARK', 1)
dcc_fcc_dlr['Park'] = dcc_fcc_dlr['PARK'].str.lower()
dcc_fcc_dlr['Location_fcc'] = dcc_fcc_dlr['Location_fcc'].str.lower()
dcc_fcc_dlr['English_Name'] = dcc_fcc_dlr['English_Name'].str.lower()
dcc_fcc_dlr['Name'] = dcc_fcc_dlr['Name'].str.lower()
dcc_fcc_dlr

In [ ]:
# merged_data[merged_data['Location_fcc'].notnull()]
## Location_fcc seems to be the superset of name since name is a specific area in location_fcc
#Thus I concatenate both into one field
dcc_fcc_dlr['Location'] = dcc_fcc_dlr['Name'].map(str)+','+dcc_fcc_dlr['Location_fcc'].map(str)
dcc_fcc_dlr[dcc_fcc_dlr['Location_fcc'].notnull()]

# ## Combine Park and Location into one column
dcc_fcc_dlr['ve'] = dcc_fcc_dlr.Park.fillna(dcc_fcc_dlr.'Location', inplace=True)
dcc_fcc_dlr['Park'] = dcc_fcc_dlr['Park'].fillna(dcc_fcc_dlr['Location'])
dcc_fcc_dlr
dcc_fcc_dlr.drop(['English_Name', 'Loc', 've', 'Irish_Name','Location_fcc','Name'], axis=1, inplace=True)
dcc_fcc_dlr.drop(['Location'],axis=1, inplace=True)
dcc_fcc_dlr = dcc_fcc_dlr.rename(columns = {'Park': 'Location'})

In [ ]:
dcc_fcc_dlr

In [ ]:
## Now we shall divide the dataset into two- one with no NAs in Latitude and Longitude and one with NAs
merged_data = dcc_fcc_dlr[dcc_fcc_dlr['Latitude'].notnull()]
lat_long_discrepant = dcc_fcc_dlr[dcc_fcc_dlr['Latitude'].isnull()]
lat_long_discrepant
# merged_data

In [ ]:
## Extracting lat long data from google api for location data

latitude = []
longitude = []
import requests
## requested an api key from google for easy access to lat long data
myapi_key = "## provide your api key ##"

url = 'https://maps.googleapis.com/maps/api/geocode/json'

## using iterrows() for all rows of lat_long_discrepant, we extract each datafield of location value and find its coordinates
for i in lat_long_discrepant.iterrows():
    ## Searching for coordinates only in Ireland since some places such as Belcamp exist in USA too
    resp_lodger = requests.get('https://maps.googleapis.com/maps/api/geocode/json?address={0}&key={1}'.format(i[1][12] + ", Ireland", myapi_key))
    ## Logging all data to json format
    respons_json = resp_lodger.json()
    ## For all ok status - that is status i have access to
    if respons_json['status'] == 'OK':
           latitude.append(respons_json['results'][0]['geometry']['location']['lat'])
           longitude.append(respons_json['results'][0]['geometry']['location']['lng'])
   



In [ ]:
## Putting values of the lists latitude and longitude into the respective columns

lat_long_discrepant['Latitude'] = latitude
lat_long_discrepant['Longitude'] = longitude
lat_long_discrepant

In [ ]:
## Finally append the two datasets merged_data and new lat_long_discrepant datasets
final_dataset = merged_data.append(lat_long_discrepant)
final_dataset

## remove duplicates
final_dataset = final_dataset.drop_duplicates(subset=list(final_dataset), keep=False)
final_dataset


### output the final dataset as a csv

final_dataset.to_csv('##saving the final output as a csv##', sep=',', encoding='utf-8')
